## Section 1: Setup and Multi-Asset Data

In [ ]:
import sys
sys.path.insert(0, '/Users/ajaiupadhyaya/Documents/Models')

from core.backtesting import SimpleMLPredictor, BacktestEngine
from models.ml import EnsemblePredictor

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Define multi-asset portfolio
assets = {
    'US Equities': ['SPY', 'QQQ', 'IWM'],      # S&P 500, Tech, Small-cap
    'Fixed Income': ['TLT', 'IEF', 'SHV'],     # Long, Intermediate, Short bonds
    'Commodities': ['GLD', 'DBC'],             # Gold, Commodity basket
    'Currencies': ['FXY', 'DXY']               # Yen, Dollar Index
}

print("Multi-Asset Portfolio Structure:")
total_assets = sum(len(tickers) for tickers in assets.values())
print(f"  Total assets: {total_assets}")
for asset_class, tickers in assets.items():
    print(f"  {asset_class}: {', '.join(tickers)}")

## Section 2: Download and Prepare Data

In [ ]:
# Download data for all assets
print("\nDownloading data for all assets...")
data = {}
all_tickers = [t for tickers in assets.values() for t in tickers]

for ticker in all_tickers:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        data[ticker] = df[['Close', 'Volume']].copy()
        print(f"{ticker}: {len(df)} days")
    except:
        print(f"{ticker}: Failed to download")

print(f"\nSuccessfully loaded {len(data)} assets")

# Get common date range
common_dates = None
for ticker, df in data.items():
    if common_dates is None:
        common_dates = set(df.index)
    else:
        common_dates = common_dates.intersection(set(df.index))

# Align all data to common dates
for ticker in data.keys():
    data[ticker] = data[ticker].loc[list(common_dates)].sort_index()

print(f"Common period: {list(common_dates)[0].date()} to {list(common_dates)[-1].date()}")
print(f"Data points per asset: {len(list(common_dates))}")

## Section 3: Calculate Returns and Correlations

In [ ]:
# Calculate returns for all assets
returns = pd.DataFrame()

for ticker, df in data.items():
    returns[ticker] = df['Close'].pct_change()

returns = returns.dropna()

# Calculate correlation matrix
corr_matrix = returns.corr()

print("Asset Class Returns Statistics:\n")
print(f"{'Asset':<8} {'Mean Daily %':<15} {'Volatility':<15} {'Sharpe':<10}")
print("-" * 48)

for ticker in sorted(returns.columns):
    mean_ret = returns[ticker].mean() * 252 * 100  # Annualized
    vol = returns[ticker].std() * np.sqrt(252) * 100  # Annualized
    sharpe = mean_ret / vol if vol > 0 else 0
    print(f"{ticker:<8} {mean_ret:>6.2f}% {vol:>14.2f}% {sharpe:>9.2f}")

# Show key correlations
print(f"\nKey Correlations:")
print(f"  SPY vs TLT: {corr_matrix.loc['SPY', 'TLT']:.3f} (stocks vs bonds)")
print(f"  SPY vs GLD: {corr_matrix.loc['SPY', 'GLD']:.3f} (stocks vs gold)")
print(f"  TLT vs GLD: {corr_matrix.loc['TLT', 'GLD']:.3f} (bonds vs gold)")
print(f"  SPY vs DXY: {corr_matrix.loc['SPY', 'DXY']:.3f} (stocks vs dollar)")

## Section 4: Train Predictors for Each Asset

In [ ]:
# Train predictors for each asset
print("Training ML predictors for each asset...\n")

predictors = {}
signals = {}

for ticker, df in data.items():
    # Prepare data with features
    df_prep = df.copy()
    df_prep['returns'] = df_prep['Close'].pct_change()
    df_prep['SMA_5'] = df_prep['Close'].rolling(5).mean()
    df_prep['SMA_20'] = df_prep['Close'].rolling(20).mean()
    df_prep['volatility'] = df_prep['returns'].rolling(20).std()
    df_prep = df_prep.dropna()
    
    # Train ensemble predictor
    predictor = EnsemblePredictor(lookback_window=20)
    predictor.train(df_prep)
    predictors[ticker] = predictor
    
    # Generate signals
    pred_signals = predictor.predict(df_prep)
    signals[ticker] = pred_signals[-len(df_prep):]
    
    print(f"{ticker}: Mean signal {np.mean(pred_signals):.4f}, Std {np.std(pred_signals):.4f}")

print(f"\nTraining complete for {len(predictors)} assets")

## Section 5: Portfolio Construction - Equal Weight

In [ ]:
# Equal weight portfolio
print("Equal Weight Portfolio Strategy\n")
print("Allocation: 1/N for each asset")

n_assets = len(data)
equal_weight = {ticker: 1.0 / n_assets for ticker in data.keys()}

print(f"\nPosition sizes ({n_assets} assets):")
for ticker, weight in sorted(equal_weight.items()):
    print(f"  {ticker}: {weight*100:.1f}%")

# Calculate portfolio returns
portfolio_returns_eq = pd.Series(0, index=returns.index)

for ticker, weight in equal_weight.items():
    portfolio_returns_eq += returns[ticker] * weight

print(f"\nEqual Weight Portfolio Performance:")
print(f"  Annualized Return: {portfolio_returns_eq.mean() * 252 * 100:.2f}%")
print(f"  Annualized Volatility: {portfolio_returns_eq.std() * np.sqrt(252) * 100:.2f}%")
print(f"  Sharpe Ratio: {(portfolio_returns_eq.mean() * 252) / (portfolio_returns_eq.std() * np.sqrt(252)):.2f}")
print(f"  Cumulative Return: {(1 + portfolio_returns_eq).prod() - 1:.2%}")

## Section 6: Portfolio Construction - Signal-Weighted

In [ ]:
# Signal-weighted portfolio (higher positions for stronger signals)
print("\nSignal-Weighted Portfolio Strategy\n")
print("Allocation: Proportional to ML signal strength")

# Get latest signals
latest_signals = {ticker: signals[ticker][-1] for ticker in signals.keys()}

# Normalize to positive (shift from [-1,1] to [0,2])
signal_weights = {}
for ticker, sig in latest_signals.items():
    signal_weights[ticker] = max(0, sig)  # Only long signals (no shorting)

# Normalize to sum to 1
total_signal = sum(signal_weights.values())
if total_signal > 0:
    signal_weights = {k: v/total_signal for k, v in signal_weights.items()}
else:
    signal_weights = {k: 1.0/len(signal_weights) for k in signal_weights.keys()}

print(f"\nSignal-Based Position Sizes:")
for ticker, weight in sorted(signal_weights.items()):
    print(f"  {ticker}: {weight*100:>5.1f}% (signal: {latest_signals[ticker]:>6.3f})")

# Calculate portfolio returns
portfolio_returns_sig = pd.Series(0, index=returns.index)

for ticker, weight in signal_weights.items():
    portfolio_returns_sig += returns[ticker] * weight

print(f"\nSignal-Weighted Portfolio Performance:")
print(f"  Annualized Return: {portfolio_returns_sig.mean() * 252 * 100:.2f}%")
print(f"  Annualized Volatility: {portfolio_returns_sig.std() * np.sqrt(252) * 100:.2f}%")
print(f"  Sharpe Ratio: {(portfolio_returns_sig.mean() * 252) / (portfolio_returns_sig.std() * np.sqrt(252)):.2f}")
print(f"  Cumulative Return: {(1 + portfolio_returns_sig).prod() - 1:.2%}")

## Section 7: Min-Variance Portfolio Optimization

In [ ]:
# Minimum variance portfolio (optimize for lowest volatility)
print("\nMinimum Variance Portfolio Strategy\n")
print("Allocation: Minimize portfolio volatility")

# Calculate covariance matrix
cov_matrix = returns.cov() * 252  # Annualized

# Simple min-variance: inverse volatility weighting
volatilities = returns.std() * np.sqrt(252)
inverse_vol = 1.0 / volatilities
min_var_weights = inverse_vol / inverse_vol.sum()

print(f"\nMin-Variance Position Sizes (inverse volatility):")
for ticker in sorted(min_var_weights.index):
    vol = volatilities[ticker] * 100
    weight = min_var_weights[ticker]
    print(f"  {ticker}: {weight*100:>5.1f}% (volatility: {vol:>6.2f}%)")

# Calculate portfolio returns
portfolio_returns_mv = pd.Series(0, index=returns.index)

for ticker in min_var_weights.index:
    portfolio_returns_mv += returns[ticker] * min_var_weights[ticker]

print(f"\nMin-Variance Portfolio Performance:")
print(f"  Annualized Return: {portfolio_returns_mv.mean() * 252 * 100:.2f}%")
print(f"  Annualized Volatility: {portfolio_returns_mv.std() * np.sqrt(252) * 100:.2f}%")
print(f"  Sharpe Ratio: {(portfolio_returns_mv.mean() * 252) / (portfolio_returns_mv.std() * np.sqrt(252)):.2f}")
print(f"  Cumulative Return: {(1 + portfolio_returns_mv).prod() - 1:.2%}")

## Section 8: Portfolio Comparison

print("\nPORTFOLIO STRATEGY COMPARISON\n")
print(f"{'Strategy':<25} {'Return':<12} {'Volatility':<12} {'Sharpe':<10}")
print("-" * 59)

portfolios = {
    'Equal Weight': portfolio_returns_eq,
    'Signal Weighted': portfolio_returns_sig,
    'Min Variance': portfolio_returns_mv
}

for name, port_returns in portfolios.items():
    ret = port_returns.mean() * 252 * 100
    vol = port_returns.std() * np.sqrt(252) * 100
    sharpe = ret / vol if vol > 0 else 0
    print(f"{name:<25} {ret:>6.2f}% {vol:>11.2f}% {sharpe:>9.2f}")

# Buy & Hold SPY
spy_return = data['SPY']['Close'].pct_change().mean() * 252 * 100
spy_vol = data['SPY']['Close'].pct_change().std() * np.sqrt(252) * 100
spy_sharpe = spy_return / spy_vol if spy_vol > 0 else 0
print(f"{'Buy & Hold SPY':<25} {spy_return:>6.2f}% {spy_vol:>11.2f}% {spy_sharpe:>9.2f}")

## Section 9: Risk Analysis

# Calculate drawdowns for each portfolio
def calculate_max_drawdown(returns):
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown.min()

print("\nRISK METRICS\n")
print(f"{'Strategy':<25} {'Max Drawdown':<15} {'Calmar Ratio':<15}")
print("-" * 55)

for name, port_returns in portfolios.items():
    max_dd = calculate_max_drawdown(port_returns)
    ret = port_returns.mean() * 252
    calmar = ret / abs(max_dd) if max_dd != 0 else 0
    print(f"{name:<25} {max_dd*100:>8.2f}% {calmar:>14.2f}")

# SPY comparison
spy_returns = data['SPY']['Close'].pct_change()
spy_max_dd = calculate_max_drawdown(spy_returns)
spy_ret = spy_returns.mean() * 252
spy_calmar = spy_ret / abs(spy_max_dd) if spy_max_dd != 0 else 0
print(f"{'Buy & Hold SPY':<25} {spy_max_dd*100:>8.2f}% {spy_calmar:>14.2f}")

## Section 10: Sector Rotation Strategy

# Sector rotation: Allocate to strongest signal in each sector
print("\nSECTOR ROTATION STRATEGY\n")
print("Allocate to strongest performer in each asset class\n")

sector_rotation = {}

for sector, tickers in assets.items():
    # Get latest signals for this sector
    sector_signals = {t: latest_signals[t] for t in tickers if t in latest_signals}
    
    if sector_signals:
        # Pick strongest signal
        best_ticker = max(sector_signals, key=sector_signals.get)
        
        # Allocate proportionally within sector
        sector_weight = 1.0 / len(assets)  # Equal sector allocation
        sector_rotation[best_ticker] = sector_weight
        
        print(f"{sector}: {' '.join(tickers)}")
        print(f"  → Selected {best_ticker} (signal: {sector_signals[best_ticker]:.3f})")
        print(f"  → Allocation: {sector_weight*100:.1f}%\n")

# Backfill missing weights as 0
for ticker in data.keys():
    if ticker not in sector_rotation:
        sector_rotation[ticker] = 0

# Calculate portfolio returns
portfolio_returns_sr = pd.Series(0, index=returns.index)

for ticker, weight in sector_rotation.items():
    if weight > 0:
        portfolio_returns_sr += returns[ticker] * weight

print(f"Sector Rotation Portfolio Performance:")
print(f"  Annualized Return: {portfolio_returns_sr.mean() * 252 * 100:.2f}%")
print(f"  Annualized Volatility: {portfolio_returns_sr.std() * np.sqrt(252) * 100:.2f}%")
print(f"  Sharpe Ratio: {(portfolio_returns_sr.mean() * 252) / (portfolio_returns_sr.std() * np.sqrt(252)):.2f}")
print(f"  Cumulative Return: {(1 + portfolio_returns_sr).prod() - 1:.2%}")

## Section 11: Implementation Roadmap

print("="*70)
print("MULTI-ASSET ML PORTFOLIO FRAMEWORK")
print("="*70)

print(f"\n✓ COMPLETED:")
print(f"  1. Multi-asset data pipeline (11 global assets)")
print(f"  2. Individual ML predictors per asset")
print(f"  3. Equal weight portfolio")
print(f"  4. Signal-weighted allocation")
print(f"  5. Minimum variance optimization")
print(f"  6. Sector rotation strategy")
print(f"  7. Risk metrics and comparisons")

print(f"\n➜ ADVANCED STRATEGIES:")
print(f"  - Efficient frontier computation (Markowitz)")
print(f"  - Black-Litterman optimization")
print(f"  - Risk parity allocation")
print(f"  - Factor-based portfolio (momentum, value, quality)")
print(f"  - Hedge ratio optimization (stocks vs bonds)")
print(f"  - Pairs trading (cointegration)")
print(f"  - Volatility targeting (constant risk)")

print(f"\n📊 MONITORING SYSTEMS:")
print(f"  - Daily rebalancing rules")
print(f"  - Drift detection (position weights vs targets)")
print(f"  - Risk alerts (drawdown, volatility spikes)")
print(f"  - Correlation breakdown alerts")
print(f"  - Signal consistency checks")
print(f"  - Model performance tracking")

print(f"\n" + "="*70)